In [33]:
## Library
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime

In [41]:
#URL & Page
URL = "https://www.blockchain.com/btc/unconfirmed-transactions"
page = requests.get(URL)

#Make it readable
soup = bs(page.content)

#Find TIME, BTC-VALUE, USD-VALUE & HASH
results_numbers = soup.find_all("span", {'class' : 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC'})
results_hash = soup.find_all("a", {'class' : 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK'})

#Make arrays of only required values
only_time = [results_numbers[i] for i in range(0, len(results_numbers), 3)]
only_BTC = [results_numbers[i] for i in range(1, len(results_numbers), 3)]
only_USD = [results_numbers[i] for i in range(2, len(results_numbers), 3)]
only_hash = [results_hash[i] for i in range(0, len(results_hash), 1)]

#Make dataframe
dataF = {'Hash' : only_hash, 'Time' : only_time, 'BTC-value' : only_BTC, 'USD-value' : only_USD}
frame = pd.DataFrame(data=dataF)

print(f"{datetime.now().hour}:{datetime.now().minute}")

#Print Dataframe
print()
print(frame)

22:26

                                                 Hash     Time  \
0   [84e9c1ac0d7449a4ee7a5e0629a014e9744c143cc5557...  [21:26]   
1   [1780e0f98f2b929a889cf532f92a49a4a043036a4bb89...  [21:26]   
2   [ebb88dc37f1577e094e92c9d0a7aff14392102b12488b...  [21:26]   
3   [c3a4a34c506c258a877d5184b21503591e0f43ded0a1d...  [21:26]   
4   [16c0167ed04e7df56597f73e7da9cd8ae496d5eff0443...  [21:26]   
5   [2e89c94486d271b98ba9596d579cecd81f46565ca6fbc...  [21:26]   
6   [098ee169a53763595cf1ecb9f34b9339962287ec3b5fd...  [21:26]   
7   [7213c6c0bd53f4e865f79486c47350c8ca4dd217a118c...  [21:26]   
8   [02f6d3d146faf91c7eda8af8b900d773cb7c42840acd0...  [21:26]   
9   [f45519f7bba1499b03d16497f378508a0f93f8a54b786...  [21:26]   
10  [cb2b4c4f234386a008a225083a6992a74e6e37dda742e...  [21:26]   
11  [0a0f236de860f6b6df7ff9db0c3b4fd77033303dae6e3...  [21:26]   
12  [384d79e41048308b23e14393798d9030046cda932054d...  [21:26]   
13  [27fe12cb973310c4b306c12dd702feea2d32330b9bd84...  [21:26]   
14 